# **Thin Plate Spline Motion Model + Real-ESRGAN Infernece**

**Credit**:

- Thin Plate Spline Motion Model - https://github.com/yoyo-nb/Thin-Plate-Spline-Motion-Model

- Real-ESRGAN Inference - https://github.com/xinntao/Real-ESRGAN

Thin Plate Spline Motion Model + Real-ESRGAN Infernece - https://github.com/RiteshK-611/TPSMM-RealESRGAN

# Setup Thin Plate Spline Motion Model

In [ ]:
!git clone https://github.com/RiteshK-611/TPSMM-RealESRGAN.git
basePath = "/content/TPSMM-RealESRGAN"
%cd {basePath}

In [ ]:
tpsmmFolder = 'Thin-Plate-Spline-Motion-Model_main'
realesrganFolder = 'Real-ESRGAN_main'
tpsmmPath = basePath + '/' + tpsmmFolder
realesrganPath = basePath + '/' + realesrganFolder

In [ ]:
%cd {tpsmmFolder}
!mkdir checkpoints
# Download Model
!wget -c https://cloud.tsinghua.edu.cn/f/da8d61d012014b12a9e4/?dl=1 -O checkpoints/vox.pth.tar
#!wget -c https://cloud.tsinghua.edu.cn/f/483ef53650b14ac7ae70/?dl=1 -O checkpoints/ted.pth.tar
#!wget -c https://cloud.tsinghua.edu.cn/f/9ec01fa4aaef423c8c02/?dl=1 -O checkpoints/taichi.pth.tar
#!wget -c https://cloud.tsinghua.edu.cn/f/cd411b334a2e49cdb1e2/?dl=1 -O checkpoints/mgif.pth.tar

In [ ]:
import torch
import os

# edit the config
device = torch.device('cuda:0')
dataset_name = 'vox' # ['vox', 'taichi', 'ted', 'mgif']
output_path = basePath + '/outputs'
source_image_path = basePath + '/inputs/source.png'
driving_video_path = basePath + '/inputs/driving.mp4'
output_video_path = basePath + '/outputs/generated.mp4'
config_path = tpsmmPath + '/config/vox-256.yaml'
checkpoint_path = tpsmmPath + '/checkpoints/vox.pth.tar'
predict_mode = 'relative' # ['standard', 'relative', 'avd']
find_best_frame = False # when use the relative mode to animate a face, use 'find_best_frame=True' can get better quality result

if not os.path.exists(output_path):
  os.makedirs(output_path)

pixel = 256 # for vox, taichi and mgif, the resolution is 256*256
if(dataset_name == 'ted'): # for ted, the resolution is 384*384
    pixel = 384

if find_best_frame:
  !pip install face_alignment

# Visualize

In [ ]:
try:
  import imageio
  import imageio_ffmpeg
except:
  !pip install imageio_ffmpeg
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
import os

warnings.filterwarnings("ignore")

source_image = imageio.imread(source_image_path)
reader = imageio.get_reader(driving_video_path)

source_image = resize(source_image, (pixel, pixel))[..., :3]

fps = reader.get_meta_data()['fps']
driving_video = []
try:
    for im in reader:
        driving_video.append(im)
except RuntimeError:
    pass
reader.close()

driving_video = [resize(frame, (pixel, pixel))[..., :3] for frame in driving_video]

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani


HTML(display(source_image, driving_video).to_html5_video())

In [ ]:
from demo import load_checkpoints
inpainting, kp_detector, dense_motion_network, avd_network = load_checkpoints(config_path = config_path, checkpoint_path = checkpoint_path, device = device)

In [ ]:
from demo import make_animation
from skimage import img_as_ubyte

if predict_mode=='relative' and find_best_frame:
    from demo import find_best_frame as _find
    i = _find(source_image, driving_video, device.type=='cpu')
    print ("Best frame: " + str(i))
    driving_forward = driving_video[i:]
    driving_backward = driving_video[:(i+1)][::-1]
    predictions_forward = make_animation(source_image, driving_forward, inpainting, kp_detector, dense_motion_network, avd_network, device = device, mode = predict_mode)
    predictions_backward = make_animation(source_image, driving_backward, inpainting, kp_detector, dense_motion_network, avd_network, device = device, mode = predict_mode)
    predictions = predictions_backward[::-1] + predictions_forward[1:]
else:
    predictions = make_animation(source_image, driving_video, inpainting, kp_detector, dense_motion_network, avd_network, device = device, mode = predict_mode)

#save resulting video
imageio.mimsave(output_video_path, [img_as_ubyte(frame) for frame in predictions], fps=fps)

HTML(display(source_image, driving_video, predictions).to_html5_video())

In [ ]:
from IPython.display import HTML
from base64 import b64encode

def show_video(video_path, video_width = 600):

  video_file = open(video_path, "r+b").read()

  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
  return HTML(f"""<video width={video_width} controls><source src="{video_url}"></video>""")

# output video
show_video(output_video_path)

# Download Animated Video

In [ ]:
from google.colab import files
# Download the results
files.download(output_video_path)

# Setup Real-ESRGAN Inference

In [ ]:
%cd ../Real-ESRGAN_main

!pip install basicsr
!pip install facexlib
!pip install gfpgan
!pip install ffmpeg-python
!pip install -r requirements.txt
!python setup.py develop
%cd ..

inputVideoFile = f'{output_path}/generated.mp4'
outputVideoFile = f'{output_path}/enhanced.mp4'

# Inference

In [ ]:
! python Real-ESRGAN_main/inference_realesrgan_video.py -i {inputVideoFile} -o {output_path} -n realesr-animevideov3 -s 2 -suffix out
 # Argements
 # -i, --input: input video
 # -i, --output: output video
 # -n, --model_name: Used model name
 # -s, --outscale: Scale
 # -suffix: Suffix of the output video

# Visualize Videos

In [ ]:
from IPython.display import HTML
from base64 import b64encode
import os

os.rename(f'{output_path}/generated_out.mp4', outputVideoFile)

html_str=""
filepaths=[inputVideoFile, outputVideoFile]

for filepath in filepaths:
  width = 500
  mp4 = open(filepath,'rb').read()
  data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
  title = "Generated" if filepath == inputVideoFile else "Enhanced"
  html_str += f"""
  <div style="text-align: center;">
    <h1>{title}</h1>
    <video width={width} controls>
          <source src={data_url} type="video/mp4">
    </video>
  </div>
  """
HTML(f"""<div style="display:flex; justify-content:space-evenly">{html_str}</div>""")

# Download Enhanced Video


In [ ]:
from google.colab import files
# Download the results
files.download(outputVideoFile)